Stone Soup Goes NUTS

In this example we will show a comparison between particle filter proposals in Stone Soup. 

The example will present the three proposal components in a single target scenario with non-linear measurement model (Cartesian To Bearing Range) in 2D for simplicity.

The three components considered are:
- Dynamics Proposal;
- Kalman Proposal;
- NUTS (No U-Turn Sampler) proposal;


First, lets load some packages.

In [6]:
import numpy as np
from scipy.stats import uniform, multivariate_normal
from datetime import datetime, timedelta

# Stone Soup components
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
    ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.state import GaussianState, ParticleState
from stonesoup.types.array import StateVector
from stonesoup.types.numeric import Probability
from stonesoup.types.particle import Particle
from stonesoup.types.detection import TrueDetection
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.types.track import Track
from stonesoup.types.hypothesis import SingleHypothesis

# Proposals
from stonesoup.proposal.nuts import NUTSProposal
from stonesoup.proposal.simple import KalmanProposal

# Kalman components
from stonesoup.predictor.kalman import UnscentedKalmanPredictor
from stonesoup.updater.kalman import UnscentedKalmanUpdater

# Tracking Components
from stonesoup.predictor.particle import ParticlePredictor
from stonesoup.updater.particle import ParticleUpdater
from stonesoup.resampler.particle import SystematicResampler

# Plotter
from stonesoup.plotter import AnimatedPlotterly

As every Stone Soup example, we have a series of steps to perform :
- create a ground-truth trajectory;
- create a set of detections from the trajectory;
- instantiate the various tracking components;
- perform the tracking loop on the various algorithms;
- visualise the final tracks and results.

In [7]:
# Lets fix a random seed for reproducibility
np.random.seed(1992)

# Simulation paramters 
simulation_steps = 100
start_time = datetime.now().replace(microsecond=0)
mapping = (0, 2)
number_particles = 100  # number of particles to consider

# Target transition model
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(1.5),
                                                          ConstantVelocity(1.5)])

# Generate ground truth
timesteps = [start_time]
truth = GroundTruthPath([GroundTruthState([1, 1, 1, 1], timestamp=timesteps[0])])

# Generate the ground truth path
for k in range(1, simulation_steps):
    timesteps.append(start_time + timedelta(seconds=k))
    truth.append(GroundTruthState(
        transition_model.function(truth[k - 1], noise=True, time_interval=timedelta(seconds=1)),
        timestamp=timesteps[k]))

# Instantiate the measurement model 
measurement_model = CartesianToBearingRange(  
    ndim_state=4,
    mapping=mapping,
    noise_covar=np.diag([np.radians(1), 5]),
    translation_offset=np.array([[0], [0]]))

# Generate the measurements
measurement_set = []
for state in truth:
    measurement = measurement_model.function(state, noise=True)
    measurement_set.append(TrueDetection(state_vector=measurement,
                                         groundtruth_path=truth,
                                         timestamp=state.timestamp,
                                         measurement_model=measurement_model))

We have prepared the ground-truth and its detections, now we can visualise them. After we will prepare the various tracking components.

In [8]:
Plotter = AnimatedPlotterly(timesteps=timesteps)

Plotter.plot_ground_truths(truth, mapping=mapping)
Plotter.plot_measurements(measurement_set, mapping=mapping)
Plotter.fig.show()

## Proposals

The proposal class enters in the `Predictor` object and guides the particle to a new state. 

### Dynamic as proposal
This is the simplest among the other proposal and uses the `Transition Model` to propagate the particles ahead in time to the new state that will be updated during the `Update step`.

### Kalman as proposal
This particular proposal employs the `Kalman Predictor` and `Kalman Updater` to propagate the particles, this particular implementation performs the Kalman loop on each singular particle, therefore we note to be particularly slower in comparison to the other, given the number of particles considered. 

### NUTS proposal
The No U-Turn Sampler proposal is a variation of Hamiltonian Monte Carlo algorithm that does not need to be tuned in the number of leap-frog integration steps since it automatically propagates the particles until a U-turn is detected. Please note that this proposal is strongly influenced by the `step-size` parameter, and the performances can be better or worse depending on the choice. Automatic stepsize (and Mass matrix) evaluation would solve this issue.

In [9]:
# Lets prepare the tracking components
resampler = SystematicResampler()
updater = ParticleUpdater(measurement_model=measurement_model,
                           resampler=resampler)

# Dynamic as proposal
dynamic_predictor = ParticlePredictor(transition_model)

# Kalman proposal
kalman_predictor_proposal = UnscentedKalmanPredictor(transition_model)
kalman_updater = UnscentedKalmanUpdater(measurement_model)

kalman_proposal = KalmanProposal(predictor=kalman_predictor_proposal,
                                 updater=kalman_updater)

kalman_predictor = ParticlePredictor(transition_model, 
                                     proposal=kalman_proposal)

# Nuts proposal
nuts = NUTSProposal(transition_model=transition_model,
         measurement_model=measurement_model,
         step_size=2,
         mass_matrix=np.eye(4),
         mapping=mapping,
         v_mapping=(1, 3),
         target_proposal_input=None,
         grad_target=None,
         num_dims=4,
         num_samples=number_particles)

nuts_predictor = ParticlePredictor(transition_model, proposal=nuts)

# priors
prior_state = GaussianState(
    StateVector([1, 1, 1, 1]),
    np.diag([10, 5, 10, 5]),
    timestamp=timesteps[0])

# Sample the particles
samples = multivariate_normal.rvs(
    np.array(prior_state.state_vector).reshape(-1),
    prior_state.covar,
    size=number_particles)

# create the particles
particles = [Particle(sample.reshape(-1, 1),
                      weight=Probability(1. / number_particles))
             for sample in samples]

# Specify the particle priors for each tracker
particle_prior1 = ParticleState(state_vector=None,
                                particle_list=particles,
                                timestamp=timesteps[0])

particle_prior2 = ParticleState(state_vector=None,
                                particle_list=particles,
                                timestamp=timesteps[0])

particle_prior3 = ParticleState(state_vector=None,
                                particle_list=particles,
                                timestamp=timesteps[0])

# prepare the various tracks
dynamic_track = Track(particle_prior1)  # dynamics
kalman_track = Track(particle_prior2)  # kalman 
nuts_track = Track(particle_prior3)  # NUTS

# Loop over the measurements
for detection in measurement_set:

    # dynamic proposal
    prediction = dynamic_predictor.predict(particle_prior1, timestamp=detection.timestamp)
    hypothesis = SingleHypothesis(prediction, detection)
    post = updater.update(hypothesis)
    dynamic_track.append(post)
    particle_prior1 = dynamic_track[-1]

    # kalman proposal
    prediction = kalman_predictor.predict(particle_prior2, 
                                          timestamp=detection.timestamp,
                                          measurement=detection)

    hypothesis = SingleHypothesis(prediction, detection)
    post = updater.update(hypothesis)
    kalman_track.append(post)
    particle_prior2 = kalman_track[-1]

    # NUTS proposal
    prediction = nuts_predictor.predict(particle_prior3, 
                                        timestamp=detection.timestamp,
                                        measurement=detection)
    hypothesis = SingleHypothesis(prediction, detection)
    post = updater.update(hypothesis)
    nuts_track.append(post)
    particle_prior3 = nuts_track[-1]

c:\Users\User\Documents\GitHub\Signal_processing\StoneSoupNUTS\vevn\lib\site-packages\stonesoup\functions\__init__.py:225: UserWarning:

LinAlgError('Matrix is not positive definite')



We have perform the tracking with the different proposals now we can visualise them.

In [10]:

Plotter.plot_tracks(dynamic_track, mapping=mapping, line=dict(color='green'), track_label='Dynamic Proposal')
Plotter.plot_tracks(kalman_track, mapping=mapping, line=dict(color='blue'), track_label='Kalman Proposal')
Plotter.plot_tracks(nuts_track, mapping=mapping, line=dict(color='orange'), track_label='NUTS Proposal')
Plotter.fig.show()